In [1]:
import numpy as np
from GibbsSampler import GibbsSampler
from DataGeneration import generate_multiple_series

### Initialize the Gibbs Sampler
Data and weights:

In [2]:
data = generate_multiple_series(150, 120, 10**4, 10**5, "gibbs.csv")
w = np.random.random(120)
w = w/np.sum(w)

Create the Gibbs sampler object:

In [3]:
gibbs = GibbsSampler(data, 16, w)

c:\Users\konto\Documents\Economy-and-Population-Growth\Priors.py:78: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mean=mean, cov=cov, size=n)


### Step 1: Calculate $\{X_i\}_{i=1}^n$

In [4]:
gibbs.step_1()